# Static Q-Balls
#### Computation with FastAI

Self-gravitating complex scalar field in flat spacetime with a self-interacting potential, $U(\Psi) = \lambda \left( \Psi^6 - a \Psi^4 + b \Psi^2 \right)$.

*Ansatz*: $\Psi = f(r)e^{i \omega t}$

*Equation of Motion*: $f'' + \dfrac{2}{r} f' + \omega^2 f - \lambda \left( 6 f^5 - 4 a f^3 + 2 b f \right) = 0$ with $f'(0) = 0$

*Contants of the problem*: $\lambda = 1~,~~a = 2~,~~b = 1.1$

##### Import packages

In [3]:
import torch as torch
import torch.utils.data as D
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

##### Select device

In [4]:
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
device

device(type='mps')

### Configuration of the problem

In [5]:
# contants of the problem
lmbd = 1
a = 2
b = 1.1

# Number of radial points
nPoints = 10000

# Number of validation points
nValid = 2000

# Number of training points
nTrain = nPoints-nValid

### Preparation of dataset and dataloaders

In [6]:
# Independent variable
r_data = torch.rand(nPoints,device=device)

# Output of the Neural Network
f_data = torch.zeros_like(r_data)

In [7]:
# Dataset with all data
dset_data = D.TensorDataset(r_data,f_data)

In [8]:
# Random Split of the dataset with all data into two for training and validation
dset_train, dset_valid = D.random_split(dset_data,[nTrain,nValid])

In [9]:
# Initializing the dataloaders for the train and valid sets
dl_train = D.DataLoader(dset_train,batch_size=100,shuffle=True)
dl_valid = D.DataLoader(dset_valid,batch_size=100,shuffle=False)

### Model

In [10]:
class Simple_net(torch.nn.Module):
  def __init__(self):
    super(Simple_net,self).__init__()
    self.fc1 = nn.Linear(1,100)
    self.fc2 = nn.Linear(100,500)
    self.fc3 = nn.Linear(500,1000)
    self.fc4 = nn.Linear(1000,1)

  def forward(self,x):
    x = F.gelu(self.fc1(x))
    x = F.gelu(self.fc2(x))
    x = F.gelu(self.fc3(x))
    x = self.fc4(x)
    return x

  def reset_weights(self):
    torch.nn.init.kaiming_uniform_(self.fc1.weight)
    torch.nn.init.kaiming_uniform_(self.fc2.weight)
    torch.nn.init.kaiming_uniform_(self.fc3.weight)
    torch.nn.init.kaiming_uniform_(self.fc4.weight)

In [11]:
model = Simple_net()
model.to(device)

Simple_net(
  (fc1): Linear(in_features=1, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=500, bias=True)
  (fc3): Linear(in_features=500, out_features=1000, bias=True)
  (fc4): Linear(in_features=1000, out_features=1, bias=True)
)